# How To: Integarted with GitHub and Azure Log Analytics

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The sample notebook retrieves Kusto query (KQL) definition file from GitHub, parses the JSON object.  Then it copies query to clipboard and opens Log Analytics window.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Retrieve Log Analytics information 
2. Retrieve KQL from GitHub and go to Log Analytics

## 1. Retrieve Log Analytics Information

In [ ]:
import json
def read_config_values(filePath):
    with open(filePath) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config["tenant_id"],
                    json_config["subscription_id"],
                    json_config["resource_group"],
                    json_config["workspace_id"],
                    json_config["workspace_name"])
    return None

path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    workspace_id = read_config_values("../config.json")[3]
    subscription_id = read_config_values("../config.json")[1]
    resource_group = read_config_values("../config.json")[2]

    ######################################################################
    workspace_name = read_config_values("../config.json")[4]
    print("Your Log Analytic Workspace: " + workspace_name)
    ######################################################################
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')

## 2. Retrieve KQL from GitHub and go to Log Analytics

In [ ]:
# Load queries from GitHub
def get_query_from_github(query_name):
    import requests
    url = "https://raw.githubusercontent.com/Azure/Orion/master/Hunting%20Queries/Deployed/{}.json".format(query_name)

    response = requests.get(url)
    response.encoding = response.apparent_encoding
    resJson = json.loads(response.text)
    if resJson != None:
        try:
            print(resJson['query'])
            return resJson['query']
        except Exception as e:
            print(e)
            
# Copy the query to Clipboard
def copy_query_to_clipboard(query_text):
    url = 'https://ms.portal.azure.com/?feature.showassettypes=Microsoft_Azure_Security_Insights_SecurityInsightsDashboard#blade/Microsoft_Azure_Security_Insights/MainMenuBlade/7/subscriptionId/{}/resourceGroup/{}/workspaceName/{}'.format(subscription_id, resource_group, workspace_name)
    html_str = (
        """<!DOCTYPE html>
        <html><body>
        <input  id="asi_demo_query" type="text" readonly style="font-weight: bold; border: none; width:1px;" size = '"""
        + str(len(query_text))
        + """' value='"""
        + query_text
        + """'>
        <a target="_new" href="javascript:void(0);" onclick="asi_demo_copy()">Copy query to clipboard and go to Log Analytics</a>
        <script>
        var asi_demo_win = null
        function asi_demo_copy() {
            var copyText = document.getElementById("asi_demo_query");
            copyText.select();
            document.execCommand("copy");

            var w = screen.width - 300;
            var h = screen.height - 300;
            params = 'width='+w+',height='+h
            asi_demo_win = window.open('"""
                    + url
                    + """', 'asi_demo_win', params);
        }

        </script>
        </body></html>"""
    )

    return html_str    

In [ ]:
from IPython.display import HTML
# file name: Create Office Account.json
HTML(copy_query_to_clipboard(get_query_from_github('Create Office Account')))